## part 1

In [1]:
from bs4 import BeautifulSoup
import numpy as np
# import concurrent.futures
# from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch, spacy,nltk,subprocess, json, requests,string,csv

model_name = "distilroberta-base"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# !python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
n_gram_range = (1, 2)
stop_words = "english"
embeddings=[]

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

import nltk, string, numpy as np
# nltk.download('punkt')

/opt/homebrew/Caskroom/mambaforge/base/envs/DT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
def get_npr_stories(p):
    # Send a GET request to the NPR API
    r = requests.get("http://api.npr.org/query?apiKey=***", params=p)

    # Parse the XML response to get the story URLs
    root = ET.fromstring(r.content)
    story_urls = [story.find('link').text for story in root.iter('story')]

    # For each story URL, send a GET request to get the HTML content
    full_stories = []
    for url in story_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content of the story. This will depend on the structure of the webpage.
        # Here, we're assuming that the main content is in a <p> tag. You might need to adjust this depending on the webpage structure.
        story = soup.find_all('p')

        # Extract the text from the story
        full_story = ' '.join(p.text for p in story)
        full_stories.append(full_story)
    return full_stories


In [ ]:
def chunk_text(text, max_len):
    # Tokenize the text into tokens
    tokens = nltk.word_tokenize(text)

    # Calculate the number of chunks and the size of the final chunk
    num_chunks = len(tokens) // max_len
    final_chunk_size = len(tokens) % max_len

    # If the final chunk is too small, distribute its tokens among the other chunks
    if final_chunk_size < max_len / 2:
        num_chunks += 1
        chunk_sizes = [len(tokens) // num_chunks + (1 if i < len(tokens) % num_chunks else 0) for i in range(num_chunks)]
        chunks = [tokens[sum(chunk_sizes[:i]):sum(chunk_sizes[:i+1])] for i in range(num_chunks)]
    else:
        chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len)]

    return chunks


In [ ]:
# from torch.utils.data import DataLoader
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoModel, AutoTokenizer
import torch

def featurize_stories(text, max_len, top_k):
    # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([text])
    all_candidates = count.get_feature_names_out()
    doc = nlp(text)
    noun_phrases = set(chunk.text.strip().lower() for chunk in doc.noun_chunks)
    nouns = set()
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.add(token.text)

    all_nouns = nouns.union(noun_phrases)
    candidates = list(filter(lambda candidate: candidate in all_nouns, all_candidates))
    candidate_tokens = tokenizer(candidates, padding=True, return_tensors="pt")
    # candidate_tokens = {k: v.to(device) for k, v in (candidate_tokens).items()}
    candidate_embeddings = model(**candidate_tokens)["pooler_output"]
    candidate_embeddings = candidate_embeddings.detach()#.to_numpy()

    # words = nltk.word_tokenize(text)
    # chunks = [words[i:i + 512] for i in range(0, len(words), 512)]
    chunks = chunk_text(text, max_len)  # use this to chunk better and use less padding thus less memory but also less affect from averging

    for chunk in chunks:
        text_tokens = tokenizer(chunk, padding=True, return_tensors="pt")
        text_tokens = {k: v.to(device) for k, v in (text_tokens).items()}
        text_embedding = model(**text_tokens)["pooler_output"]
        text_embedding = text_embedding.detach()#.to_numpy()
        embeddings.append(text_embedding)
    max_emb_shape = max(embedding.shape[0] for embedding in embeddings)
    padded_embeddings = [np.pad(embedding.cpu(), ((0, max_emb_shape - embedding.shape[0]), (0, 0))) for embedding in embeddings]
    avg_embedding = np.min(padded_embeddings, axis=0)
    distances = cosine_similarity(avg_embedding, candidate_embeddings.cpu())
    torch.cuda.empty_cache()
    return [candidates[index] for index in distances.argsort()[0][::-1][-top_k:]]



In [ ]:
import pandas as pd
# data=pd.read_csv('/content/drive/MyDrive/consult/Louie_disaster_tweets.csv',header=None)
data=pd.read_csv('florida-hurricane-tweet.csv')

In [ ]:
data.to_parquet('8.5k_florida_tweets.parquet')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features.txt',sep='\t')

In [ ]:
rank_articles=[]
from tqdm import tqdm
for i in tqdm(range(len(df),len(data['text']))):
    try:
        cc=featurize_stories(data['text'][i], max_len=512, top_k=4)
        # print(cc)
        rank_articles.append(cc)
    except IndexError:
        pass

In [ ]:
flattened_list = [item for sublist in rank_articles for item in sublist]
from collections import Counter
counter = Counter(flattened_list)
df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])

df = df.sort_values(by='Count',ascending=False)
df.to_csv('/content/drive/MyDrive/consult/florida-hurricane-tweet_features2.txt',sep='\t')
print(len(df))
# df[:25]

In [ ]:
filtered_list = [sublist for sublist in rank_articles if any('fire'.lower() in s.lower() for s in sublist)]
filtered_list

In [ ]:
import pandas as pd
df = pd.read_csv('florida-hurricane-tweet_features.txt',sep='\t')
df2=pd.read_csv('florida-hurricane-tweet_features2.txt',sep='\t')
df=pd.concat([df,df2])
print(df['Unnamed: 0'])
df = df.groupby('Unnamed: 0').sum().sort_values(by='Count',ascending=False)
df=df[df['Count']>int(np.round(len(df)*.001))]

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

# nouns = ['apple', 'John', 'London', 'dog', 'Mary', 'Paris', 'banana']
nouns= df.reset_index()['Unnamed: 0'].to_list()
doc = nlp(' '.join(nouns))

proper_nouns = [token.text for token in doc if token.pos_ == 'PROPN']

print(proper_nouns)

In [ ]:
print(len(proper_nouns))
proper_nouns

In [ ]:
len(nouns)

In [ ]:
i=['Iceland', '16-01-2024', ['Lava from a volcanic eruption in southwestern Iceland has streamed into a nearby town, engulfing homes and forcing the evacuation of local residents.\n', 'The Icelandic Meteorological Office said the eruption began just before 8:00 a.m. local time Sunday about a half mile from the town of Grindavík after a series of intense earthquakes.\n', 'A second fissure opened after noon Sunday and sent lava flows into the town, officials said.\n', 'Iceland\'s president, Guðni Th. Jóhannesson, said in an address to the nation on Sunday that a "daunting period of upheaval" had begun for those in the Reykjanes peninsula.\n', '"We continue to hope for as good an outcome as possible, in the face of these tremendous forces of nature," he said.\n', 'This is the second time in a month that a volcano has erupted just outside Grindavík, a coastal town about 25 miles from the Icelandic capital of Reykjavík.\n', 'An eruption on December 18 sent lava spewing into the air, but residents had already been told to leave due to heightened seismic activity.\n', "Iceland's government said Sunday that Grindavík had been evacuated early that morning and that the eruption isn't expected to reach other populated areas.\n", 'So far no flights have been disrupted, and officials are monitoring threats to infrastructure. At least three homes have either burned down or been overtaken by lava, according to the Icelandic broadcaster RUV.\n', "The government also said the eruption doesn't present a threat to life.\n", 'But in his speech, Jóhannesson offered his sympathy to the loved ones of Lúðvík Pétursson, a man who went missing in a work accident in Grindavík last week.\n', "According to Sky News, the 50-year-old Pétursson was filling crevasses formed by volcanic activity and earthquakes when he fell in a crack that had opened after last month's eruption.\n", 'Iceland is a hotspot for seismic activity, with 32 active volcanoes. A volcano erupts roughly every five years in the country, though eruptions have occurred more frequently recently. \n', ' Copyright 2024 NPR. To see more, visit https://www.npr.org.  ', '\n'], None, None, None]

## part 2

In [3]:
import argparse, signal,datetime,subprocess,json,os
from dotenv import load_dotenv
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-p', type=int, default=1, help='Parallelize requests')
# parser.add_argument('-s', type=datetime, default=20230101, help='start date')
# parser.add_argument('-e', type=datetime, default=20231231, help='end date')
args, unknown = parser.parse_known_args()
load_dotenv()

True

In [4]:

os_url=([os.environ['OS_TOKEN']])
n=10

bash_command = f"""
curl -X GET "{os_url}" -H 'Content-Type: application/json' -d '{{
"_source": ["metadata.GDELT_DATE", "metadata.page_title","metadata.DocumentIdentifier", "metadata.Organizations","metadata.Persons","metadata.Themes","metadata.text", "metadata.Locations"],
    "size": {n},
    "query": {{
        "bool": {{
            "must": [
                {{"match_all": {{}}}}
            ]
        }}
    }}
}}'
"""
process = subprocess.run(bash_command, shell=True, capture_output=True, text=True)
output = process.stdout
data = json.loads(output)
data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
output

''

In [18]:
# data = json.loads("input/feat_input.json")
with open("input/feat_input.json", 'r') as f:
    data = json.load(f)
data['_scroll_id']

'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoBRZuY2RmVWxjc1NseUlYODE0MlZYQ3pnAAAAAAAApzUWMjJMTDE0ck9RZ3k2OHJkc1A5MVhMQRZNN19sZmw3MFFZTzVJZzNqTzRRVnp3AAAAAAAANsIWY2xMdUdIeU5TYTJLNGpYOHozTk9PQRYxTEVzc2lyb1RCeVVMUTFpRFp0ZW5nAAAAAAAAXpAWRE9yYlNHbndTT2VrbkxRMzc0U3UtdxZKU0Vsa0t3bVQzS2lsUmgyQW9ub0xRAAAAAAAAVxQWalVQTFFvdDJTZ3VzNV9oa2hiZ0VwdxZnUEVPLXNyZlJndXRZMnltVm5IdEpnAAAAAAAAV64WeHhuSHpZbUVRQi1SVGFmeDJ6Nm9mQQ=='

In [20]:
len(data['hits']['hits'])

100

In [ ]:
# Keep track of the scroll ID
scroll_id = response['_scroll_id']
es = OpenSearch([os.environ['OS_TOKEN']])
# Keep scrolling until no more results
while len(response['hits']['hits']):
    # print(response['hits']['hits'])  # process results

    response = es.scroll(
        scroll_id=scroll_id,
        scroll='1m'  # length of time to keep the scroll window open
    )

    # Update the scroll ID
    scroll_id = response['_scroll_id']


In [ ]:
from dots_feat import featurize_stories, process_data, get_data, process_hit
data = get_data(10)
data['hits']['hits']

In [ ]:
import sys,os,argparse,datetime
sys.argv = ['dots_feat.py', '-n', '10', '-f', '3', '-o', 'dots_feats.csv', '-p', '1']#, '-s', '20230101', '-e', '20231231']
parser = argparse.ArgumentParser(description='Process OS data for dynamic features.')
parser.add_argument('-n', type=int, default=10, help='Number of data items to get')
parser.add_argument('-f', type=int, default=3, help='Number of features per item to get')
parser.add_argument('-o', type=str, default='dots_feats.csv', help='Output file name')
parser.add_argument('-p', type=int, default=1, help='Parallelize requests')
# parser.add_argument('-s', type=datetime.date, default=datetime.datetime.strptime(20230101), help='start date')
# parser.add_argument('-e', type=datetime.date, default=datetime.datetime.strptime(20231231), help='end date')
args, unknown = parser.parse_known_args()

from DOTS.dots_feat import featurize_stories, process_data, get_data, process_hit

In [ ]:
import DOTS

In [ ]:
s="20230101"
e="20231231"

s_date = datetime.datetime.strptime(s, "%Y%m%d")
e_date = datetime.datetime.strptime(e, "%Y%m%d")

# Convert datetime objects to ISO 8601 format
s_iso = s_date.isoformat() + "Z"
e_iso = e_date.isoformat() + "Z"

In [ ]:
[s_iso,e_iso]

In [ ]:
def get_data(n=args.n):
    bash_command = f"""
    curl -X GET "{os_url}" -H 'Content-Type: application/json' -d '{{
"_source": ["metadata.GDELT_DATE", "metadata.page_title","metadata.DocumentIdentifier", "metadata.Organizations","metadata.Persons","metadata.Themes","metadata.text", "metadata.Locations"],
        "size": {n},
        "query": {{
            "bool": {{
                "must": [
                    {{"match_all": {{}}}},
                ]
            }}
        }}
    }}'
    """
    process = subprocess.run(bash_command, shell=True, capture_output=True, text=True)
    output = process.stdout
    data = json.loads(output)
    return data

In [ ]:
data = get_data(10)
data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data = get_data(10)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data = get_data(10)
hits = data['hits']['hits']
hit = hits[1]
hit['_source']

from datetime import datetime
source = hit['_source']
date = datetime.strptime(source['metadata']['GDELT_DATE'], "%Y%m%d%H%M%S")
loc = source['metadata']['Locations']
# ex = None #source['metadata']['Extras']
title = source['metadata']['page_title']
url = source['metadata']['DocumentIdentifier']

In [ ]:
s = source['metadata']['Locations']

s = s.replace("'", '"')  # json requires double quotes for keys and string values
list_of_dicts = json.loads(s)


# location_full_name = source['metadata']['Locations']['Location FullName']

for dict in list_of_dicts:
    if 'Location FullName' in dict:
        print(dict['Location FullName'])

In [ ]:
data

In [ ]:

def process_data_fast(data):
    articles = []
    hits = data['hits']['hits']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_hit, hits))

    with open('output.csv', 'w') as file:
        writer = csv.writer(file)
        for text, soup,date,loc,title in results:
            articles.append([loc,loc,loc,date,title,title,text])
            # articles.append(text)

            writer.writerow([loc,loc,loc,date,title,title,text]) # force location into top feature, also assume title has important info
            # writer.writerow(text)
            writer.writerow(['\n'])

    with open('output.html', 'w') as file:
        file.write(str(results[-1][1]))  # Write the soup of the last hit

    # with open('output.csv', 'w') as file:
    #     file.write(str(results[-1][0]))  # Write the z of the last hit

    return articles


In [ ]:
from dots_feat import process_data_fast

In [ ]:
def process_hits(hit):
    source = hit['_source']
    GDELT_DATE = source['metadata']['GDELT_DATE']
    page_title = source['metadata']['page_title']
    url = source['metadata']['DocumentIdentifier']
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to get {url}")
        return z
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all(['p','div','a'])
    if not paragraphs:
        print(f"No <p> tags in {url}")
        return z

    for p in paragraphs:
        z.append(p.get_text())
    return z

with open('output.html', 'w') as file:
    file.write(str(soup))
    
with open('output.csv', 'w') as file:
    file.write(str(z))

# return articles
articles=[]
hits = data['hits']['hits']
for hit in hits:
    z=process_hits(hit)
    articles.append(z)
    

In [ ]:

def process_hit(hit):
    z = []
    source = hit['_source']
    GDELT_DATE = source['metadata']['GDELT_DATE']
    page_title = source['metadata']['page_title']
    location = source['metadata']["Locations"]
    extras = source['metadata']["Extras"]
    url = source['metadata']['DocumentIdentifier']

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for p in soup.find_all('p'):
        z.append(p.get_text())
    z.append(page_title)
    return z, soup

def process_data_fast(data):
    articles = []
    hits = data['hits']['hits']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_hit, hits))

    with open('output.csv', 'w') as file:
        writer = csv.writer(file)
        for z, soup in results:
            writer.writerow(z)
            writer.writerow(['\n'])
            articles.append(z)

    with open('output.html', 'w') as file:
        file.write(str(results[-1][1]))  # Write the soup of the last hit

    # with open('output.csv', 'w') as file:
    #     file.write(str(results[-1][0]))  # Write the z of the last hit

    return articles


In [ ]:
data

In [ ]:
data = get_data(n=10)
# data = get_big_data()
# articles = process_data(data)
import concurrent.futures
articles = process_data_fast(data)

In [ ]:
rank_articles=[]
for i in articles[1:]:
    try:
        cc=featurize_stories(str(i), max_len=512, top_k=1)
        rank_articles.append(cc)
    except IndexError:
        pass

In [ ]:
with open('dots_feats.csv', 'w') as file:
    file.write(rank_articles)